# **Capstone Project - Benefit Zones for Indian Restaurants**
## **Prakirth Govardhanam**
## **Applied Data Science Capstone by IBM/Coursera**

## Introduction/Business-Problem
In this project, I try to find possible-beneficial locations within the Neighborhoods (Districts) of Helsinki, Finland, for establishing a chain of Indian Restaurants. The conditions to fulfill in order are:

* CONDITION 1 - Distance from Popularity Centre (Assumption) in the Neighborhood (District) - for popularity
* CONDITION 2 - Absence of other Indian restaurants in the Neighborhood - to limit competition


## Data
Data sources used to determine the Neighborhoods within the city of Helsinki are provided by:

* Wikipedia_(https://en.wikipedia.org/wiki/Names_of_places_in_Finland_in_Finnish_and_in_Swedish#Municipalities)_ - for listing the Neighborhoods (Districts) of Helsinki
* The City of Helsinki(https://kartta.hel.fi/avoindata) - for geospatial Data
* Foursquare API - for popular venues, restaurants and their respective geospatial data

## Project Assumption
**_Popularity Centre_** = the centroid of the top-10 venues (filtered by ratings) in each District will be considered as the "popularity centre" within every District

# PART 1 - Data Preparation

## PART 1.1 - Data Extraction

### Import necessary libraries

In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import geocoder
from geopy.geocoders import Nominatim

### Clarification:
* Names of anything in Finland has its name in 2 languages, Finnish & Swedish
* Hence, names of Districts are also in same pattern: Finnish-name (Swedish-name)
### ***Assumption #1***
* In the current extracted labels data, Finnish-names are Available for every place where as **"Swedish-names of Finland _could be_ confused with Swedish-names of Sweden" in the FourSquare API**.
* Hence, we will extract and work only with Finnish-names of the Neighbourhoods & Districts


In [2]:
#url with Helsinki District names
url = 'https://en.wikipedia.org/wiki/Names_of_places_in_Finland_in_Finnish_and_in_Swedish#Municipalities'

#parsing the webpage for html content
html = requests.get(url).text
soup = BeautifulSoup(html, features='html.parser')

#extract <a href> tags
atags = soup.select('a[href]')

#extract titles of <a href> tags
titles = []
for atag in atags:
    titles.append(atag.get('title'))

#slice the labels of Helsinki Districts
districts = titles[titles.index('Ala-Malmi'): titles.index('Ylä-Malmi')+1]
print(f"Total Districts listed: {len(districts)}")

Total Districts listed: 110


In [3]:
#extract coordinates from District/Neighborhood names using geopy.geocoders.Nominatim
geolocator = Nominatim(user_agent='Helsinki_districts')

#empty lists for latitude & longitude values and None values, if any
lats = []
longs = []

#looping through district names for coordinates
for name in districts:
    location = geolocator.geocode(name)
    try:
        lats.append(location.latitude)
        longs.append(location.longitude)
    except AttributeError:
        pass

In [4]:
print(f"Total values identified \n(Latitude, Longitude): {len(lats), len(longs)}")

Total values identified 
(Latitude, Longitude): (109, 109)


## PART 1.2 - Investigating Data for errors

### Districts with ***None*** values for coordinates

In [5]:
# Investigating None value in districts list, if Any
trial = []
for name in districts:
    location = geolocator.geocode(name)
    try:
        trial.append(location.latitude)
    except AttributeError as err:
        print('None value detected!')
        raise

None value detected!


AttributeError: 'NoneType' object has no attribute 'latitude'

In [8]:
#Identify District with NoneType coordinate
print(f"District with NoneType coordinate:\n{districts[len(trial)]}")

District with NoneType coordinate:
Kampinmalmi


In [9]:
#Direct verification 
geolocator.geocode('Kampinmalmi').latitude

AttributeError: 'NoneType' object has no attribute 'latitude'

### Districts with Improper coordinates (Detected *Manually*)

In [55]:
wrong_coords = ['Pasila','Töölö']
for name in wrong_coords:
    print(f"Locations as identified by geopy.geocoders API for {name}:\n{geolocator.geocode(name)}\n")

Locations as identified by geopy.geocoders API for Pasila:
Brasil

Locations as identified by geopy.geocoders API for Töölö:
Toolo, Loroum, Nord, Burkina Faso



In [12]:
#Districts, Latitudes & Longitudes with NoneType & Improper coordinates - to be removed from Lists

print(f"BEFORE Cleaning:\nTotal Districts:{len(districts)}\nTotal Latitude values:{len(lats)}\nTotal Longitude values:{len(longs)}")

loc_to_pop = ['Pasila','Töölö','Kampinmalmi']
lat_to_pop = [-10.3333333, 13.744717]
long_to_pop = [-53.2, -1.9645989]

#Remove districts without coordinates and with improper coordinates
for loc in loc_to_pop:
    districts.remove(loc)

#Remove improper coordinates    
for lat, long in zip(lat_to_pop, long_to_pop):
    lats.remove(lats[lats.index(lat)])
    longs.remove(longs[longs.index(long)])
    
print(f"\nAFTER Cleaning:\nTotal Districts:{len(districts)}\nTotal Latitude values:{len(lats)}\nTotal Longitude values:{len(longs)}")

BEFORE Cleaning:
Total Districts:110
Total Latitude values:109
Total Longitude values:109

AFTER Cleaning:
Total Districts:107
Total Latitude values:107
Total Longitude values:107


In [13]:
#Frame all extracted values in a Dataframe
districts_df = pd.DataFrame(data= zip(districts, lats, longs), columns=['District', 'Latitude', 'Longitude'])
districts_df.head()

,District,Latitude,Longitude
0,Ala-Malmi,60.249474,25.014539
1,Alppiharju,60.189728,24.944120
2,Aurinkolahti,60.201507,25.155669
3,Eira,60.156191,24.938375
4,Etelä-Haaga,60.211615,24.891092


In [14]:
districts_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107 entries, 0 to 106
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   District   107 non-null    object 
 1   Latitude   107 non-null    float64
 2   Longitude  107 non-null    float64
dtypes: float64(2), object(1)
memory usage: 2.6+ KB


# PART 2 - Exploratory Data Analysis

1. plot map of city of Helsinki Districts using Folium
2. Use FourSquare API to:
* extract popular(top 10) venues around each District
* locate _Indian-restaurants_ present in the District
* locate **"_Popularity centres_"** by calculating the centroid of the top-10 venues from each district using clustering-methods, such as linkage, fcluster..
3. plot map of **"_Popularity centres_"** & _Indian-restuaurants_ using Folium
4. plot Districts with **"_Popularity centres_"**:
* **without Indian-restaurants**, labeled as **"_Benefit-Zones_"** *(in green)*
* **with Indian-restaurants NOT IN top-10 venues**, labeled as **_Minor Competition-Zones_** *(in blue)*
* **with Indian-restaurants IN top-10 venues**, labeled as **_Major Competition-Zones_** *(in red)*

## PART 2.1 - Plot city map of Helsinki indicating Districts

### Import necessary libraries

In [15]:
import folium

In [16]:
address = 'Helsinki, Finland'
geolocator = Nominatim(user_agent='Helsinki_district_map')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(f"Coordinates of Helsinki are: {latitude}, {longitude}")

Coordinates of Helsinki are: 60.1674881, 24.9427473


In [59]:
helsinki_map = folium.Map(location=[latitude, longitude], zoom_start=6)

for dist, lat, long in zip(districts_df.District, districts_df.Latitude, districts_df.Longitude):
    label = folium.Popup('{}'.format(dist), parse_html=True)
    folium.CircleMarker([lat, long],
    radius=20,
    popup=label,
    fill=False,
    parse_html=False).add_to(helsinki_map)

helsinki_map

### Districts with Improper coordinates (Outside Helsinki, *detected using Folium map*)

In [79]:
# Verification of locations with improper coordinates' Districts
wrong_districts = ['Vanhakaupunki','Siltasaari', 'Reijola', 'Vironniemi', 'Koivusaari']

for district in wrong_locs:
    print(f"District as identified by geopy.geocoders API for {district}:\n{geolocator.geocode(district)}\n")

District as identified by geopy.geocoders API for Vanhakaupunki:
Gamla stan, Stortorget, Gamla stan, Södermalms stadsdelsområde, Stockholm, Stockholms kommun, Stockholms län, 111 29, Sverige

District as identified by geopy.geocoders API for Siltasaari:
Siltasaari, Jyränkö, Heinola, Lahden seutukunta, Päijät-Häme, Etelä-Suomen aluehallintovirasto, Manner-Suomi, Suomi

District as identified by geopy.geocoders API for Reijola:
Reijola, Joensuu, Joensuun seutukunta, Pohjois-Karjala, Itä-Suomen aluehallintovirasto, Manner-Suomi, 80330, Suomi

District as identified by geopy.geocoders API for Vironniemi:
Vironniemi, Siilinjärvi, Kuopion seutukunta, Pohjois-Savo, Itä-Suomen aluehallintovirasto, Manner-Suomi, 71870, Suomi

District as identified by geopy.geocoders API for Koivusaari:
Koivusaari, Nurmes, Pielisen Karjalan seutukunta, Pohjois-Karjala, Itä-Suomen aluehallintovirasto, Manner-Suomi, Suomi



In [92]:
#collecting indices of rows to be removed
rows_to_pop = []
for district in wrong_districts:
    index = districts_df.loc[districts_df.District == district].index.to_list()
    rows_to_pop.append(index)

indices = [j for i in rows_to_pop for j in i]
indices = sorted(indices)
print(f"Indices to be removed from the districts_df Dataframe: {indices}")

Indices to be removed from the districts_df Dataframe: [30, 84, 92, 103, 105]


In [102]:
#Drop rows in districts_df Dataframe
districts_df.drop(indices, axis=0, inplace=True)
districts_df.reset_index(drop=True, inplace=True)
print(f"Dataframe refined for venues extraction from FourSquare API:\nTotal Rows: {districts_df.shape[0]}\nTotal Columns: {districts_df.shape[1]}")

Dataframe refined for venues extraction from FourSquare API:
Total Rows: 102
Total Columns: 3


In [159]:
#Map corrected for wrong districts
helsinki_map = folium.Map(location=[latitude, longitude], zoom_start=6)

for dist, lat, long in zip(districts_df.District, districts_df.Latitude, districts_df.Longitude):
    label = folium.Popup('{}'.format(dist), parse_html=True)
    folium.CircleMarker([lat, long],
    radius=20,
    popup=label,
    fill=False,
    parse_html=False).add_to(helsinki_map)

helsinki_map

## PART 2.2 - Use **FourSquareAPI** & Extract Venues

### ***Assumption #2 (Important)***
* In reality, there are more Indian Restaurants than **explored Indian Restaurants using *FourSquare API***
* Since, the project is based on **"using FourSquare API for implementation of the Idea"** we will assume the following:
    * **"explored Indian Restaurants" _=_ "existing Indian Restaurants"** 


In [111]:
#Credentials
CLIENT_ID = 'CXC1D1CNWMCS54XHC3M0VLPRLBCPQQMID0OZC04Z0VYTMSAU' 
CLIENT_SECRET = 'OQRFM1BNLVMREJ3N3VJBAWGKU2ERVDEBC3Q1M2UXHBVNDBN3' 
VERSION = '20201201' 
LIMIT = 100

In [112]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, long in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = f"https://api.foursquare.com/v2/venues/explore?&client_id={CLIENT_ID}&client_secret={CLIENT_SECRET}&v={VERSION}&ll={lat},{long}&radius={radius}&limit={LIMIT}"
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            long, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['District', 
                  'District Latitude', 
                  'District Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [113]:
helsinki_venues = getNearbyVenues(districts_df.District, districts_df.Latitude, districts_df.Longitude)

Ala-Malmi
Alppiharju
Aurinkolahti
Eira
Etelä-Haaga
Haaga
Hakaniemi
Hakuninmaa
Haltiala
Heikinlaakso
Hermanni (Helsinki)
Herttoniemen teollisuusalue
Herttoniemenranta
Herttoniemi
Hevossalmi
Hietalahti, Helsinki
Itä-Pakila
Itä-Pasila
Itäsaaret
Jollas, Helsinki
Kaarela
Kaartinkaupunki
Kaisaniemi
Kaivopuisto
Kallahti
Kallio
Keski-Pasila
Keski-Vuosaari
Kivihaka
Kluuvi
Konala
Koskela
Kruununhaka
Kulosaari
Kumpula
Kurkimäki
Kuusisaari
Laajasalo
Laakso
Länsi-Herttoniemi
Länsi-Pakila
Länsi-Pasila
Lassila
Lauttasaari
Lehtisaari, Helsinki
Malmi, Helsinki
Marttila, Helsinki
Marjaniemi
Maunula
Maunulanpuisto
Maununneva
Meilahti
Mellunkylä
Meri-Rastila
Merihaka
Metsälä
Munkkiniemi
Munkkisaari
Munkkivuori
Mustavuori
Mustikkamaa–Korkeasaari
Myllypuro
Niemenmäki
Niinisaari
Oulunkylä
Pajamäki
Pakila
Patola, Helsinki
Pihlajamäki
Pihlajisto
Pikku Huopalahti
Pirkkola
Pitäjänmäen teollisuusalue
Pitäjänmäki
Pohjois-Haaga
Pohjois-Pasila
Puistola
Pukinmäki
Punavuori
Puotila
Puotinharju
Puroniitty
Rastila
Reima

In [114]:
print(f"Total Rows:{helsinki_venues.shape[0]}, Total Columns:{helsinki_venues.shape[1]}")
helsinki_venues.head()

Total Rows:1807, Total Columns:7


,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ala-Malmi,60.249474,25.014539,Ravintola Makalu,60.250291,25.012946,Himalayan Restaurant
1,Ala-Malmi,60.249474,25.014539,Fitness24Seven,60.251597,25.013711,Gym / Fitness Center
2,Ala-Malmi,60.249474,25.014539,Malmin Uimahalli | Fix Liikuntakeskus,60.251131,25.016400,Pool
3,Ala-Malmi,60.249474,25.014539,Thai Thai,60.248500,25.010685,Thai Restaurant
4,Ala-Malmi,60.249474,25.014539,Alko,60.251465,25.013255,Liquor Store


In [129]:
print(f"Total unique Venue categories: {helsinki_venues['Venue Category'].nunique()}")

Total unique Venue categories: 261


### ***Assumption #3***
* Total Venue Category with "Indian Restaurant" = 8
* Total Venue Category with "Himalayan Restaurant" = 12 
* Hence, we will be considering **BOTH Venue Categories (Indian & Himalayan) as Indian Restaurants**

In [154]:
helsinki_indian_venues = helsinki_venues[(helsinki_venues['Venue Category'] == 'Indian Restaurant') | (helsinki_venues['Venue Category']=='Himalayan Restaurant')]
print(f"Total number of Indian Restaurants in all Helsinki Districts: {len(helsinki_indian_venues)}")
print("Districts with Indian Restaurant/s:\n")
helsinki_indian_venues

Total number of Indian Restaurants in all Helsinki Districts: 20
Districts with Indian Restaurant/s:



,District,District Latitude,District Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Ala-Malmi,60.249474,25.014539,Ravintola Makalu,60.250291,25.012946,Himalayan Restaurant
61,Aurinkolahti,60.201507,25.155669,Ravintola Mayur,60.201398,25.149978,Himalayan Restaurant
100,Etelä-Haaga,60.211615,24.891092,Roseway,60.207981,24.886940,Indian Restaurant
199,Herttoniemenranta,60.189238,25.029584,Ravintola Mantra,60.186781,25.030365,Himalayan Restaurant
229,Herttoniemi,60.195525,25.029063,Gurkha,60.194930,25.028670,Himalayan Restaurant
265,"Hietalahti, Helsinki",60.162768,24.927331,Aangan,60.163198,24.927786,Himalayan Restaurant
576,Kallahti,60.200809,25.138395,Ravintola New Light,60.205113,25.135951,Indian Restaurant
791,Konala,60.238550,24.846065,Mother India,60.239230,24.849760,Indian Restaurant
917,Länsi-Herttoniemi,60.209119,25.040027,Sunkosi,60.206577,25.042706,Himalayan Restaurant
956,Lassila,60.231027,24.876722,Ravintola Moksha,60.229298,24.880959,Indian Restaurant


### 

In [165]:
print(f"Total unique Districts with Indian restaurants: {helsinki_indian_venues.District.nunique()}")

Total unique Districts with Indian restaurants: 20


### DONE
* 20 Districts **WITH "1" INDIAN Restaurant**
* 82 Districts **WIHOUT "1" INDIAN Restaurant**
* 20 Districts - Red Zone
* 82 Districts - Green Zone

### TO-DO:
* Extract Top10 Venues from each District (method from Lab)
* Calculate _Popularity Centre_ for each District (clustering methods)
* Plot _Popularity Centre_ in Map for each District
* Plot Red Zone & Green Zone _Popularity Centre_ in Helsinki City Map